In [ ]:
#   Copyright 2025 UKRI-STFC
#   Copyright 2025 University of Manchester

#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at

#   http://www.apache.org/licenses/LICENSE-2.0

#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.
#
# Authors:
# Kubra Kumrular, University of Southampton, μ-VIS X-ray Imaging Centre 
# Evangelos Papoutsellis,  Finden Ltd
# Laura Murgatroyd (UKRI-STFC) 


In [ ]:
import numpy as np
import tifffile as tiff
import time
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing
from cil.utilities.display import show2D, show_geometry

import xml.etree.ElementTree as ET
import threading


from cil.framework import AcquisitionGeometry, AcquisitionData
from cil.framework import ImageData, ImageGeometry
from cil.processors import Slicer, AbsorptionTransmissionConverter, TransmissionAbsorptionConverter,CentreOfRotationCorrector,Binner,Normaliser, Padder
from cil.utilities.display import show2D, show_geometry 
from cil.recon import FDK
from cil.plugins.tigre import FBP # check if FDK works with 2D

from cil.io import TIFFWriter, RAWFileWriter


import matplotlib as mpl

mpl.rcParams["image.cmap"] = "inferno"

### Diondo DataReader

Diondo data reader tested on 
 - Example raw data: https://zenodo.org/records/17167576
 - Recon: https://zenodo.org/records/17167562

1) Read centre slice and reconstruct (with/without COR)
2) Read all projections and reconstruct
3) Read specific angles


- TODOs
  - Check different range values CIL vs Diondo
  - Check strange line in sinogram 2D (maybe use
https://github.com/TomographicImaging/CIL-Demos/blob/flux_normaliser_demo/misc/flux_normaliser.ipynb)
  - Test the case of specific angles option
  - Check COR of CIL vs COR of Diondo
 



In [ ]:
from readers.DiondoDataReader import DiondoDataReader

### Path to data and metadata

In [ ]:
        
path    = r'20240508_d5_3917_AV_trustfireAAA_overv'
xml_file_name    = r'20240508_d5_3917_AV_trustfireAAA_overv.xml'
proj_file_name = r'20240508_d5_3917_AV_trustfireAAA_overv'


### Create DiondoDataReader

In [ ]:
data_reader = DiondoDataReader(xml_file=xml_file_name, 
                               projection_path=proj_file_name)

### Read Centre Slice (2D)

In [ ]:
t1 = time.time()
# no joblib
data2D = data_reader.read_centre_slice()
print(f" It took {time.time() - t1} sec for single slice")

### AcqGeom 2D info

In [ ]:
ag2D = data2D.geometry
ig2D = ag2D.get_ImageGeometry()
print(f"data2D dtype is {data2D.dtype}")
print(ag2D)

In [ ]:
show_geometry(ag2D)

### Recon 2D

In [ ]:
data_absorbed2D = TransmissionAbsorptionConverter(min_intensity=1e-6)(data2D)
data_absorbed2D -= np.mean(data_absorbed2D.array[0:50,0:50])
fdk_recon2D = FDK(data_absorbed2D, image_geometry=ig2D).run()

In [ ]:
data_absorbed2D = TransmissionAbsorptionConverter()(data2D)
data_absorbed2D -= np.mean(data_absorbed2D.array[0:10,0:10])
plt.imshow(data_absorbed2D.array)


In [ ]:
plt.imshow(fdk_recon2D.array)

### Correct CoR (in 2D centre slice)

In [ ]:
data_absorbed2D = CentreOfRotationCorrector.image_sharpness()(data_absorbed2D)

In [ ]:
fdk_recon2D_cor = FDK(data_absorbed2D, image_geometry=ig2D).run()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].imshow(fdk_recon2D.array)
axs[0].set_title('CIL without CoR', fontsize=20)

axs[1].imshow(fdk_recon2D_cor.array)
axs[1].set_title('CIL with CoR', fontsize=20)

plt.tight_layout()

plt.show()

### Read Full Data

### Read all projections/angles

In [ ]:
t1 = time.time()
data3D_all_fast = data_reader.read(fread=True)
print(f" With Joblib: {time.time() - t1} sec for {data3D_all_fast.shape[0]}")

t1 = time.time()
data3D_all_slow = data_reader.read(fread=False)
print(f" Without Joblib: {time.time() - t1} sec for {data3D_all_slow.shape[0]}")

### AcqGeom 3D info

In [ ]:
ag3D = data3D_all_fast.geometry
ig3D = ag3D.get_ImageGeometry()
print(f"data3D fast dtype is {data3D_all_fast.dtype}")
print(f"data3D slow dtype is {data3D_all_slow.dtype}")
print(ag3D)

In [ ]:
show_geometry(ag3D)

### Recon 3D with COR corr

In [ ]:
background      = data3D_all_fast.get_slice(vertical=10, force=True).as_array().mean()
data3D_norm          = data3D_all_fast/background
data3D_lb            = TransmissionAbsorptionConverter(min_intensity=1e-6)(data3D_norm)
data3D_no_outring = data3D_lb - np.mean(data3D_lb.array[0:100,:,0:50])
data3D_no_outring = CentreOfRotationCorrector.image_sharpness()(data3D_no_outring)

In [ ]:
print(data3D_no_outring.geometry)

In [ ]:
print(f"CIL offset after COR is -0.03750837")

tree = ET.parse(xml_file_name)
root = tree.getroot()
recon = root.find("Recon")

print(f"Diondo offset from metadata is {float(recon.find("ProjectionCenterOffsetX").text)}")


In [ ]:
fdk_recon3D_cor = FDK(data3D_no_outring, image_geometry=ig3D).run()

In [ ]:
plt.imshow(data3D_no_outring.array[1250//2])

In [ ]:
plt.imshow(fdk_recon3D_cor.array[1500])
plt.colorbar()

In [ ]:
with open(r"20240508_d5_3917_AV_trustfireAAA_overv_800x800x2270x32bit.raw", "rb") as f:
    tmp = np.fromfile(f, dtype="float32", count=800*800*2270)

In [ ]:
diondo_recon = tmp.reshape(2270,800,800)

In [ ]:
plt.imshow(fdk_recon3D_cor.array[1500])

In [ ]:
plt.imshow(diondo_recon[1738])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].imshow(fdk_recon3D_cor.array[1500])
axs[0].set_title('CIL', fontsize=20)

axs[1].imshow(diondo_recon[1738])
axs[1].set_title('Diondo', fontsize=20)

plt.tight_layout()
# plt.subplots_adjust(wspace=-0.6, hspace=0.2)

# plt.savefig("cil_vs_zeiss_roi_recon.png")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

im0 = axs[0].imshow(fdk_recon3D_cor.array[1500, 400:500, 200:400])
axs[0].set_title('CIL', fontsize=20)
cbar0 = fig.colorbar(im0, ax=axs[0], shrink=0.9)

im1 = axs[1].imshow(diondo_recon[1738, 400:500, 200:400])
axs[1].set_title('Diondo', fontsize=20)
cbar1 = fig.colorbar(im1, ax=axs[1], shrink=0.9)

plt.tight_layout()
plt.show()


### Issue with different contrast, acqdata

In [ ]:
plt.imshow(data3D_all_fast.get_slice(vertical=0, force=True).as_array())
plt.colorbar()

In [ ]:
plt.imshow(np.mean(data3D_all_fast.array, axis=1))
plt.colorbar()

### Read some angles (user input) -> 625 out of 1250

In [ ]:
t1 = time.time()
data3D_some_angles1 = data_reader.read(angles = np.linspace(0, 360, 1250//2, endpoint=False), fread=True)
print(f" With Joblib: {time.time() - t1} sec for {data3D_some_angles1.shape[0]} projs")


In [ ]:
background      = data3D_some_angles1.get_slice(vertical=10, force=True).as_array().mean()
data3D_norm          = data3D_some_angles1/background
data3D_lb            = TransmissionAbsorptionConverter(min_intensity=1e-6)(data3D_norm)
data3D_no_outring = data3D_lb - np.mean(data3D_lb.array[0:100,:,0:50])
data3D_no_outring = CentreOfRotationCorrector.image_sharpness()(data3D_no_outring)



In [ ]:
fdk_recon1 = FDK(data3D_no_outring, image_geometry=ig3D).run()

In [ ]:
plt.imshow(fdk_recon1.array[1000], cmap="inferno")
plt.colorbar()

### Read some angles (user input) -> 312 out of 1250

In [ ]:
t1 = time.time()
data3D_some_angles2 = data_reader.read(angles = np.linspace(0, 360, 312, endpoint=False), fread=True)
print(f" With Joblib: {time.time() - t1} sec for {data3D_some_angles2.shape[0]} projs")


In [ ]:
background      = data3D_some_angles2.get_slice(vertical=10, force=True).as_array().mean()
data3D_norm          = data3D_some_angles2/background
data3D_lb            = TransmissionAbsorptionConverter(min_intensity=1e-6)(data3D_norm)
data3D_no_outring = data3D_lb - np.mean(data3D_lb.array[0:100,:,0:50])
data3D_no_outring = CentreOfRotationCorrector.image_sharpness()(data3D_no_outring)


In [ ]:
fdk_recon2 = FDK(data3D_no_outring, image_geometry=ig3D).run()

In [ ]:
plt.imshow(fdk_recon.array[1000], cmap="inferno")
plt.colorbar()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].imshow(fdk_recon1.array[1000])
axs[0].set_title('CIL 625 angles', fontsize=20)

axs[1].imshow(fdk_recon2.array[1000])
axs[1].set_title('CIL 312 angles', fontsize=20)

plt.tight_layout()
# plt.subplots_adjust(wspace=-0.6, hspace=0.2)

# plt.savefig("cil_vs_zeiss_roi_recon.png")
plt.show()

In [ ]:
# ### test
# raw_files = sorted(glob.glob(os.path.join(proj_file_name, "*.raw")))

# print(len(raw_files)), 
# print(np.linspace(0, 360, 625, endpoint=False))

# # list_ang = np.linspace(0, 360, 1250, endpoint=False)
# list_ang = np.linspace(0, 360, 625, endpoint=False)
# print(len(list_ang))

# for i in range(0,len(raw_files)-1,2):
#     print(i, raw_files[i], list_ang[i])

### Test FluxNormaliser (old cil env)

In [ ]:
from cil.processors import FluxNormaliser

In [ ]:
processor = FluxNormaliser(roi={'horizontal':(0, 50)}, target='mean')
processor.set_input(data2D)
data_norm = processor.get_output()

In [ ]:
processor.preview_configuration()

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(np.mean(data_norm.array[:,0:50], axis=1), label="After Flux")
plt.plot(np.mean(data2D.array[:,0:50], axis=1), label="Before Flux")
plt.grid("on")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].imshow(fdk_recon1.array[1000])
axs[0].set_title('CIL 625 angles', fontsize=20)

axs[1].imshow(fdk_recon2.array[1000])
axs[1].set_title('CIL 312 angles', fontsize=20)

plt.tight_layout()
# plt.subplots_adjust(wspace=-0.6, hspace=0.2)

# plt.savefig("cil_vs_zeiss_roi_recon.png")
plt.show()

In [ ]:
plt.imshow(data3D.array)